In [262]:
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score

In [263]:
import nltk
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/shreyanshsourabh/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [264]:
print(stopwords.words("english"))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [265]:
import numpy as np
import pandas as pd

# Data Pre-processing

In [266]:
train_df = pd.read_csv("/Users/shreyanshsourabh/Documents/ML projects/newsFakeorReal/train.csv")
test_df = pd.read_csv("/Users/shreyanshsourabh/Documents/ML projects/newsFakeorReal/test.csv")
sub = pd.read_csv("/Users/shreyanshsourabh/Documents/ML projects/newsFakeorReal/submit.csv")

In [267]:
train_df[1495:1496]

,id,title,author,text,label
1495,1495,Attack of the Clones translated into Chinese a...,Sausage Machine,Next Swipe left/right Attack of the Clones tra...,1


In [268]:
train_df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [269]:
test_df.head()

,id,title,author,text
0,20800,"Specter of Trump Loosens Tongues, if Not Purse...",David Streitfeld,"PALO ALTO, Calif. — After years of scorning..."
1,20801,Russian warships ready to strike terrorists ne...,NaN,Russian warships ready to strike terrorists ne...
2,20802,#NoDAPL: Native American Leaders Vow to Stay A...,Common Dreams,Videos #NoDAPL: Native American Leaders Vow to...
3,20803,"Tim Tebow Will Attempt Another Comeback, This ...",Daniel Victor,"If at first you don’t succeed, try a different..."
4,20804,Keiser Report: Meme Wars (E995),Truth Broadcast Network,42 mins ago 1 Views 0 Comments 0 Likes 'For th...


In [270]:
train_df.shape

(20800, 5)

In [271]:
train_df.describe()

,id,label
count,20800.000000,20800.000000
mean,10399.500000,0.500625
std,6004.587135,0.500012
min,0.000000,0.000000
25%,5199.750000,0.000000
50%,10399.500000,1.000000
75%,15599.250000,1.000000
max,20799.000000,1.000000


In [272]:
train_df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [273]:
test_df.isnull().sum()

id          0
title     122
author    503
text        7
dtype: int64

In [274]:
train_df = train_df.fillna("")
test_df = test_df.fillna("")

In [275]:
train_df["content"] = train_df["author"]+" " + train_df["title"]
test_df["content"] = test_df["author"]+" " + test_df["title"]

In [276]:
train_df.head()

,id,title,author,text,label,content
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,Darrell Lucus House Dem Aide: We Didn’t Even S...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,"Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo..."
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,Consortiumnews.com Why the Truth Might Get You...
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,Jessica Purkiss 15 Civilians Killed In Single ...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,Howard Portnoy Iranian woman jailed for fictio...


In [277]:
X = train_df.drop("label" , axis=1)
Y = train_df['label']
print(X.shape,Y.shape)

(20800, 5) (20800,)


# Stemming

In [278]:
port_stem = PorterStemmer()

In [279]:
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]',' ',content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content

In [280]:
train_df['content'] = train_df['content'].apply(stemming)
test_df['content'] = test_df['content'].apply(stemming)

In [281]:
X = train_df["content"]
Y = train_df["label"]

X_test = test_df["content"]
Y_test = sub["label"]

# Vectorizing

In [282]:
# converting textual data to numeric data
vectorizer = TfidfVectorizer(ngram_range=(3,3))

vectorizer.fit(X)
X = vectorizer.transform(X)


X_test = vectorizer.transform(X_test)

In [283]:
print(X.shape,Y.shape,X_test.shape,Y_test.shape)

(20800, 144926) (20800,) (5200, 144926) (5200,)


# Training model

In [284]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
model.fit(X,Y)

LogisticRegression()

In [285]:
# from sklearn import svm

# model = svm.SVC(kernel='sigmoid')
# model.fit(X,Y)

# Evaluation

In [286]:
train_prediction = model.predict(X)
training_accuracy = accuracy_score(train_prediction, Y)
training_accuracy

0.8582692307692308

In [287]:
test_prediction = model.predict(X_test)
testing_accuracy = accuracy_score(test_prediction, Y_test)
testing_accuracy

0.7325